In [1]:
import pandas as pd
import ast
import json

In [2]:
import openai
from dotenv import load_dotenv

from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

import sys
import os
from pathlib import Path
import openai
from dotenv import load_dotenv
try:
    # If running as a script, use __file__ to find the directory
    __file__
    base_path = os.path.dirname(os.path.dirname(os.path.dirname(__file__)))
    sys.path.append(base_path)
except NameError:
    # If running in an interactive environment (e.g., Jupyter Notebook)
    base_path = Path().resolve().parent.parent
    sys.path.append(str(base_path))
  
load_dotenv()
model="gpt-4o"

from utils.llm import call_llm
# local
# filepath = "/home/UNT/ae0589/Desktop/HPCC/AutomaticWorkflowGeneration/ActionEngine/db/api_info/"

# Cloud
filepath = "/home/cc/AutomaticWorkflowGeneration/ActionEngine/db/api_info/"


filename = filepath + 'api_information.json'

def read_json_to_dict(filename):
    api_info = []
    with open(filename, 'r') as file:
        for line in file:
            api_info.append(json.loads(line))

    return api_info

def escape_json(text: str) -> str:
    return text.replace('{', '{{').replace('}', '}}')

api_info = read_json_to_dict(filename)
# Extract specific fields
extracted_data = [
    {
        'name': item['name'],
        'summary': item['summary']
    }
    for item in api_info
]

from pydantic import BaseModel, Field
from typing import List
class API(BaseModel):
    name: str = Field(description="Name of the API that is used in the generated user request")
    parameters: List[str] = Field(description="List of required parameters for the selected API")

class Data(BaseModel):
    id: int = Field(description="Id number for the information")
    user_request: str = Field(description="The user request that is going to require to use 2 API")
    selected_apis: List[API] = Field(description="List of APIs and their parameters used in the generated user request")

class TestData(BaseModel):
    test_data: List[Data] = Field(description="List of user requests and those related parameters")

api_info = read_json_to_dict(filename)


/home/cc/miniconda3/envs/ml/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## Generate 1-2 Easy Dataset

### Generate 2 nodes output

In [12]:

SYSTEM_PROMPT = '''
    Let's think step by step. You will be given list of API information.
    Please generate 5 different user request that is going to require to use 2 API, and extract the user request, selected APIs, and each required parameter for those selected APIs.
    Below is just an example be creative and make sure to use different APIs to make these data varieties like send_email too. 
    Output Example
    {
        user_request: "I have a creative project where I need to generate a unique image in a charcoal art style based on a specific theme and then resize the generated image to specific dimensions for a digital presentation. The theme for the image is 'A serene sunset over a quiet village'. After the image is generated, I need it to be resized to a width of 1920 pixels and a height of 1080 pixels."
        selected_apis:[
            [
                "name": "tti_Charcoal_Art"
                "parameters": ["prompt"]
            ],
            [
                "name": "tti_Charcoal_Art"
                "parameters": ["image_file", "width", "height"]
            ]

    },

    '''

USER_PROMPT = f"""
        API_information: {extracted_data}
    """
response = call_llm(TestData, "TestData" ,escape_json(SYSTEM_PROMPT), escape_json(USER_PROMPT))
response

{'test_data': [{'id': 1,
   'user_request': "I want to create a unique animation art piece based on the theme of 'Space Exploration' and then convert the generated animation into a PDF document for sharing.",
   'selected_apis': [{'name': 'tti_Animation_Art', 'parameters': ['prompt']},
    {'name': 'image_to_pdf_conversion', 'parameters': ['image_file']}]}]}

In [45]:
inter_list = {"test_data": inter_list}
with open('./query_answer_1-2_nodes.json', 'w') as json_file:
    json.dump(response, json_file, indent=4)

## Generate 3-5 nodes Inter test data

In [35]:
# 3-5 nodes


inter_list = []

### 3 nodes

In [36]:
SYSTEM_PROMPT = '''
    Let's think step by step. You will be given list of API information.
    Please generate 4 user request that is going to use exactly 3 APIs, and extract the user request, selected APIs, and each required parameter for those selected APIs.
    Make sure the user request is very specific in details as if what's kind of image you want to generat and etc.
    
    '''
USER_PROMPT = f"""
        API_information: {extracted_data}
    """

response = call_llm(TestData, "TestData" ,escape_json(SYSTEM_PROMPT), escape_json(USER_PROMPT))
inter_list.append(response)
print(inter_list)
print(len(inter_list))

[{'test_data': [{'id': 1, 'user_request': 'I want to create a digital art piece of a futuristic cityscape, enhance its quality, and then convert it to a PDF for a presentation.', 'selected_apis': [{'name': 'tti_Digital_Art', 'parameters': ['prompt']}, {'name': 'Image_Enhancement', 'parameters': ['image_file']}, {'name': 'image_to_pdf_conversion', 'parameters': ['image_file']}]}, {'id': 2, 'user_request': 'Please generate a charcoal art image of a serene forest, resize it to 1920x1080 pixels, and convert it to a PDF for printing.', 'selected_apis': [{'name': 'tti_Charcoal_Art', 'parameters': ['prompt']}, {'name': 'Image_Resizing', 'parameters': ['image_file', 'width', 'height']}, {'name': 'image_to_pdf_conversion', 'parameters': ['image_file']}]}, {'id': 3, 'user_request': 'I need a pastel art image of a sunset over the ocean, enhance its quality, and then send it via email to my colleague.', 'selected_apis': [{'name': 'tti_Pastel_Art', 'parameters': ['prompt']}, {'name': 'Image_Enhance

### 4 nodes

In [37]:
SYSTEM_PROMPT = '''
    Let's think step by step. You will be given list of API information.
    Please generate 3 user request that is going to use exactly 4 APIs, and extract the user request, selected APIs, and each required parameter for those selected APIs.
    Make sure the user request is very specific in details as if what's kind of image you want to generat and etc.
    '''
USER_PROMPT = f"""
        API_information: {extracted_data}
    """

response = call_llm(TestData, "TestData" ,escape_json(SYSTEM_PROMPT), escape_json(USER_PROMPT))
inter_list.append(response)
print(inter_list)
print(len(inter_list))

[{'test_data': [{'id': 1, 'user_request': 'I want to create a digital art piece of a futuristic cityscape, enhance its quality, and then convert it to a PDF for a presentation.', 'selected_apis': [{'name': 'tti_Digital_Art', 'parameters': ['prompt']}, {'name': 'Image_Enhancement', 'parameters': ['image_file']}, {'name': 'image_to_pdf_conversion', 'parameters': ['image_file']}]}, {'id': 2, 'user_request': 'Please generate a charcoal art image of a serene forest, resize it to 1920x1080 pixels, and convert it to a PDF for printing.', 'selected_apis': [{'name': 'tti_Charcoal_Art', 'parameters': ['prompt']}, {'name': 'Image_Resizing', 'parameters': ['image_file', 'width', 'height']}, {'name': 'image_to_pdf_conversion', 'parameters': ['image_file']}]}, {'id': 3, 'user_request': 'I need a pastel art image of a sunset over the ocean, enhance its quality, and then send it via email to my colleague.', 'selected_apis': [{'name': 'tti_Pastel_Art', 'parameters': ['prompt']}, {'name': 'Image_Enhance

### 5 nodes

In [38]:
SYSTEM_PROMPT = '''
    Let's think step by step. You will be given list of API information.
    Please generate 3 user request that is going to use exactly 5 APIs, and extract the user request, selected APIs, and each required parameter for those selected APIs.
    Make sure the user request is very specific in details as if what's kind of image you want to generat and etc.
    '''
USER_PROMPT = f"""
        API_information: {extracted_data}
    """

response = call_llm(TestData, "TestData" ,escape_json(SYSTEM_PROMPT), escape_json(USER_PROMPT))
inter_list.append(response)
print(inter_list)
print(len(inter_list))

[{'test_data': [{'id': 1, 'user_request': 'I want to create a digital art piece of a futuristic cityscape, enhance its quality, and then convert it to a PDF for a presentation.', 'selected_apis': [{'name': 'tti_Digital_Art', 'parameters': ['prompt']}, {'name': 'Image_Enhancement', 'parameters': ['image_file']}, {'name': 'image_to_pdf_conversion', 'parameters': ['image_file']}]}, {'id': 2, 'user_request': 'Please generate a charcoal art image of a serene forest, resize it to 1920x1080 pixels, and convert it to a PDF for printing.', 'selected_apis': [{'name': 'tti_Charcoal_Art', 'parameters': ['prompt']}, {'name': 'Image_Resizing', 'parameters': ['image_file', 'width', 'height']}, {'name': 'image_to_pdf_conversion', 'parameters': ['image_file']}]}, {'id': 3, 'user_request': 'I need a pastel art image of a sunset over the ocean, enhance its quality, and then send it via email to my colleague.', 'selected_apis': [{'name': 'tti_Pastel_Art', 'parameters': ['prompt']}, {'name': 'Image_Enhance

In [39]:
inter_list = {"test_data": inter_list}
with open('./test_queries/query_answer_3-5_nodes.json', 'w') as json_file:
    json.dump(inter_list, json_file, indent=4)

## Generate 6-10 Nodes Hard Date

In [36]:
# 6-10 nodes


inter_list = []

### 6 nodes

In [37]:
SYSTEM_PROMPT = '''
    Let's think step by step. You will be given list of API information.
    Please generate 2 user request that is going to use exactly 6 APIs, and extract the user request, selected APIs, and each required parameter for those selected APIs.
    Make sure the user request is very specific in details as if what's kind of image you want to generat and etc.
    
    This is just an example but learn from it and be creative to make request:
    "I am preparing a multi-media portfolio for a project that showcases urban evolution and different art styles. First, I need a futuristic animation-style image representing a city skyline of 2080 with flying vehicles and skyscrapers. For the second part, I want a charcoal-style artwork showing an old, historic part of the same city, possibly including narrow streets and ancient architecture. After generating these images, I need the futuristic animation image to be resized and enhanced for better quality. Finally, I want both the futuristic and charcoal images combined into a single PDF, along with their descriptions. Please email the final PDF to my professor for review."
    '''
USER_PROMPT = f"""
        API_information: {extracted_data}
    """

response = call_llm(TestData, "TestData" ,escape_json(SYSTEM_PROMPT), escape_json(USER_PROMPT))
inter_list.extend(response['test_data'])
print(response['test_data'])
print(len(inter_list))

[{'id': 1, 'user_request': 'I am creating a digital art gallery that showcases different art styles of a single subject. First, I need a digital art image of a serene mountain landscape during sunrise. Next, I want a pastel art image of the same mountain landscape but during sunset. Then, I need a sketching art image of the mountain landscape at night with a starry sky. After generating these images, I need the digital art image to be resized to 1920x1080 pixels and enhanced for better quality. Finally, I want all three images combined into a single PDF for presentation purposes. Please email the final PDF to my client for review.', 'selected_apis': [{'name': 'tti_Digital_Art', 'parameters': ['prompt']}, {'name': 'tti_Pastel_Art', 'parameters': ['prompt']}, {'name': 'tti_Sketching_Art', 'parameters': ['prompt']}, {'name': 'Image_Resizing', 'parameters': ['image_file', 'width', 'height']}, {'name': 'Image_Enhancement', 'parameters': ['image_file']}, {'name': 'image_to_pdf_conversion', '

### 7nodes

In [38]:
SYSTEM_PROMPT = '''
    Let's think step by step. You will be given list of API information.
    Please generate 2 user request that is going to use exactly 7 APIs, and extract the user request, selected APIs, and each required parameter for those selected APIs.
    Make sure the user request is very specific in details as if what's kind of image you want to generat and etc.
    
    This is just an example but learn from it and be creative to make request:
    "I’m working on a multi-media portfolio for a project that reflects the contrast between future and past cityscapes. First, I need to generate two images: one should be a futuristic animation of a city skyline in the year 2080 with neon lights and flying cars, and the second image should depict the same city but in a charcoal art style, focusing on its historic and ancient parts. After that, I want the futuristic animation image to be enhanced for quality, resized to fit specific dimensions, and converted to a new format. Then, I want both images combined into a single PDF document. Finally, I need the PDF to be emailed to my professor for review."
    '''
USER_PROMPT = f"""
        API_information: {extracted_data}
    """

response = call_llm(TestData, "TestData" ,escape_json(SYSTEM_PROMPT), escape_json(USER_PROMPT))
inter_list.extend(response['test_data'])
print(response['test_data'])
print(len(inter_list))

[{'id': 1, 'user_request': 'I am creating a digital art gallery that showcases different art styles of a single subject, a serene mountain landscape. First, I need to generate five images of the mountain landscape in different art styles: digital art, pastel art, ukiyo-e art, graffiti art, and mosaic art. After that, I want to enhance the quality of the digital art image, resize it to 1920x1080 pixels, and convert it to a PNG format. Finally, I need all five images combined into a single PDF document and emailed to my art curator for review.', 'selected_apis': [{'name': 'tti_Digital_Art', 'parameters': ['prompt']}, {'name': 'tti_Pastel_Art', 'parameters': ['prompt']}, {'name': 'tti_Ukiyo-e_Art', 'parameters': ['prompt']}, {'name': 'tti_Graffiti_Art', 'parameters': ['prompt']}, {'name': 'tti_Mosaic_Art', 'parameters': ['prompt']}, {'name': 'Image_Enhancement', 'parameters': ['image_file']}, {'name': 'Image_Resizing', 'parameters': ['image_file', 'width', 'height']}, {'name': 'image_form

### 8 nodes

In [39]:
SYSTEM_PROMPT = '''
    Let's think step by step. You will be given list of API information.
    Please generate 2 user request that is going to use exactly 8 APIs, and extract the user request, selected APIs, and each required parameter for those selected APIs.
    Make sure the user request is very specific in details as if what's kind of image you want to generat and etc.
    
    This is just an example but learn from it and be creative to make request:
    I’m creating a multi-media portfolio that contrasts futuristic and historical cityscapes. First, I need to generate two images: one should be a futuristic city skyline for the year 2080 in an animation style, and the second should depict the historic parts of the same city but rendered in a charcoal art style. After generating these images, I want the futuristic animation image to be enhanced, resized, and converted to a new format. Then, I need both images combined into a single PDF with descriptions. Finally, I want this PDF to be password-protected and emailed to my professor for review.
    '''
USER_PROMPT = f"""
        API_information: {extracted_data}
    """

response = call_llm(TestData, "TestData" ,escape_json(SYSTEM_PROMPT), escape_json(USER_PROMPT))
inter_list.extend(response['test_data'])
print(response['test_data'])
print(len(inter_list))

[{'id': 1, 'user_request': 'I am working on a project that involves creating a series of images depicting a futuristic city in various art styles. First, I need to generate four images: one in animation style, one in digital art style, one in graffiti art style, and one in mosaic art style. After generating these images, I want the digital art image to be enhanced and resized. Then, I need all four images to be converted into a single PDF file. Finally, I want this PDF to be password-protected and emailed to my project supervisor for feedback.', 'selected_apis': [{'name': 'tti_Animation_Art', 'parameters': ['user prompt']}, {'name': 'tti_Digital_Art', 'parameters': ['user prompt']}, {'name': 'tti_Graffiti_Art', 'parameters': ['user prompt']}, {'name': 'tti_Mosaic_Art', 'parameters': ['user prompt']}, {'name': 'Image_Enhancement', 'parameters': ['image file']}, {'name': 'Image_Resizing', 'parameters': ['image file', 'width', 'height']}, {'name': 'image_to_pdf_conversion', 'parameters': 

### 9 nodes

In [40]:
SYSTEM_PROMPT = '''
    Let's think step by step. You will be given list of API information.
    Please generate 2 user request that is going to use exactly 9 APIs, and extract the user request, selected APIs, and each required parameter for those selected APIs.
    Make sure the user request is very specific in details as if what's kind of image you want to generat and etc.

    This is just an example but learn from it and be creative to make request:
    I’m developing a multi-media portfolio that contrasts the future and the past of urban landscapes. First, I need three different images: one should be a futuristic city skyline for the year 2080 in an animation style, the second should depict the historic parts of the same city in charcoal style, and the third should be a digital art image representing the city's present-day state. After generating these images, I want the futuristic image to be enhanced and resized, and the present-day digital art image converted to a different format. Next, I need all three images combined into a single password-protected PDF with descriptions for each image. Finally, I want this PDF to be emailed to my professor for review.
    '''
USER_PROMPT = f"""
        API_information: {extracted_data}
    """

response = call_llm(TestData, "TestData" ,escape_json(SYSTEM_PROMPT), escape_json(USER_PROMPT))
inter_list.extend(response['test_data'])
print(response['test_data'])
print(len(inter_list))

[{'id': 1, 'user_request': 'I am creating a digital art gallery showcasing different art styles of a single subject, a modern cityscape. First, I need five different images: one in animation style, one in charcoal style, one in digital art style, one in futuristic art style, and one in graffiti art style. After generating these images, I want the digital art image to be enhanced and resized. Finally, I need all five images combined into a single PDF with descriptions for each image, and this PDF should be emailed to my art professor for feedback.', 'selected_apis': [{'name': 'tti_Animation_Art', 'parameters': ['prompt']}, {'name': 'tti_Charcoal_Art', 'parameters': ['prompt']}, {'name': 'tti_Digital_Art', 'parameters': ['prompt']}, {'name': 'tti_Futuristic_Art', 'parameters': ['prompt']}, {'name': 'tti_Graffiti_Art', 'parameters': ['prompt']}, {'name': 'Image_Enhancement', 'parameters': ['image_file']}, {'name': 'Image_Resizing', 'parameters': ['image_file', 'width', 'height']}, {'name'

### 10 nodes

In [41]:
SYSTEM_PROMPT = '''
    Let's think step by step. You will be given list of API information.
    Please generate 2 user request that is going to use exactly 10 APIs, and extract the user request, selected APIs, and each required parameter for those selected APIs.
    Make sure the user request is very specific in details as if what's kind of image you want to generat and etc.
    
    This is just an example but learn from it and be creative to make request:
    I'm working on a multi-media portfolio that explores the evolution of a city across four distinct eras. I need four images, each representing a different time period: (1) a futuristic city in 2080 in animation style, (2) the historic parts of the same city in charcoal style, (3) a present-day cityscape in digital art, and (4) an artistic pastel-style representation of the city during the early 1900s. After generating these images, I want the futuristic and pastel images to be enhanced and resized to match specific dimensions. Then, I need all four images combined into a single password-protected PDF with descriptions for each image. Finally, the PDF should be emailed to my professor for review.
    '''
USER_PROMPT = f"""
        API_information: {extracted_data}
    """

response = call_llm(TestData, "TestData" ,escape_json(SYSTEM_PROMPT), escape_json(USER_PROMPT))
inter_list.extend(response['test_data'])
print(response['test_data'])
print(len(inter_list))

[{'id': 1, 'user_request': "I'm creating a digital art project that showcases different art styles of a single subject, a serene mountain landscape. I need images in the following styles: (1) a vibrant animation style, (2) a detailed charcoal art style, (3) a modern digital art style, (4) a futuristic art style, (5) a graffiti art style, (6) a mosaic art style, (7) a classic painting art style, (8) a soft pastel art style, (9) a sketching art style, and (10) a traditional Japanese ukiyo-e art style. After generating these images, I want them all combined into a single PDF file for presentation purposes.", 'selected_apis': [{'name': 'tti_Animation_Art', 'parameters': ['prompt']}, {'name': 'tti_Charcoal_Art', 'parameters': ['prompt']}, {'name': 'tti_Digital_Art', 'parameters': ['prompt']}, {'name': 'tti_Futuristic_Art', 'parameters': ['prompt']}, {'name': 'tti_Graffiti_Art', 'parameters': ['prompt']}, {'name': 'tti_Mosaic_Art', 'parameters': ['prompt']}, {'name': 'tti_Painting_Art', 'par

In [42]:
inter_list = {"test_data": inter_list}
with open('./test_queries/query_answer_6-10_nodes.json', 'w') as json_file:
    json.dump(inter_list, json_file, indent=4)